In [2]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode
'''
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor

sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split() #空格分词
word_list = list(set(word_list)) #去重建立list
word_dict = {w: i for i, w in enumerate(word_list)} #词:index字典
number_dict = {i: w for i, w in enumerate(word_list)}#index:词字典
n_class = len(word_dict)

# TextRNN Parameter
batch_size = len(sentences)
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]] #返回一个index列表
        target = word_dict[word[-1]]#目标index
        print(input)
        input_batch.append(np.eye(n_class)[input]) #batch添加独热编码的list
        target_batch.append(target)#添加目标
    print(input_batch)
    return input_batch, target_batch

# to Torch.Tensor
input_batch, target_batch = make_batch(sentences)
input_batch = Variable(torch.Tensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch)) #输入Variable化



[5, 1]
[5, 4]
[5, 0]
[array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0.]]), array([[0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0.]])]


In [5]:

class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()

        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden) #输入->隐藏层
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype)) #隐藏层->输出层的映射
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))#偏置

    def forward(self, hidden, X):
        X = X.transpose(0, 1) # X : [n_step, batch_size, n_class] #转置
        outputs, hidden = self.rnn(X, hidden)
        # outputs : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        outputs = outputs[-1] # [batch_size, num_directions(=1) * n_hidden]
        model = torch.mm(outputs, self.W) + self.b # model : [batch_size, n_class]
        return model

model = TextRNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(5000):
    optimizer.zero_grad()

    # hidden : [num_layers * num_directions, batch, hidden_size]
    hidden = Variable(torch.zeros(1, batch_size, n_hidden))
    # input_batch : [batch_size, n_step, n_class]
    output = model(hidden, input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

input = [sen.split()[:2] for sen in sentences]

# Predict
hidden = Variable(torch.zeros(1, batch_size, n_hidden))

predict = model(hidden, input_batch).data.max(1, keepdim=True)#取可能性最高的
print(predict)
#print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])



Epoch: 1000 cost = 0.084195
Epoch: 2000 cost = 0.016925
Epoch: 3000 cost = 0.006358
Epoch: 4000 cost = 0.002977
Epoch: 5000 cost = 0.001546
torch.return_types.max(
values=tensor([[7.4979],
        [7.2162],
        [4.8145]]),
indices=tensor([[2],
        [6],
        [3]]))
